<a href="https://colab.research.google.com/github/MarlonRF/CPGIF_Colab/blob/master/EUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procedimentos
Vou estabelecer a regra que o conjunto de dados ficarão em apenas um arquivo (workbook|pasta) e separados por edição|semestre ficaram em planilhas separadas no mesmo arquivo. 

**Exemplo:** 
* Arquivo: Notas do EUF 
* Planilhas: 2020-1, 2019-2, etc (edições)

Referencias das edições é sempre da realização do evento, não o semestre alvo.

As inscrições para ingresso em 2020-2 acontecem em 2020-1, portanto, os dados serão agrupados como "2020-1"


In [23]:
'''-  BIBLIOTECAS NÃO NATIVAS - PRECISAM VER INSTALADAS VIA !pip -'''
!pip install fuzzywuzzy
!pip install python-Levenshtein #requisito, deixa fuzzy mais rapido
!pip install fuzzymatcher



  Created wheel for metaphone: filename=Metaphone-0.6-cp36-none-any.whl size=13906 sha256=84139757e5ca8012f327e39bb3448bca0434e3304eb17054f91c32925b53b599
  Stored in directory: /root/.cache/pip/wheels/4e/7c/f7/162d726fc83491ef23c7a0b989005024eb83a1408c96f32eaf
Successfully built metaphone


In [1]:
!git clone https://marlonfaria@hotmail.com:alien51@github.com/MarlonRF/CPGIF_Colab:repo


Cloning into 'repo'...
fatal: unable to access 'https://marlonfaria@hotmail.com:alien51@github.com/MarlonRF/CPGIF_Colab:repo/': Port number ended with 'a'


In [3]:
'''
BIBLIOTECAS
-----------
'''

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from google.colab import auth
import pandas as pd
import matplotlib.pyplot as plt
import re

'''
Para acesso ao GOOGLE DOCS
--------------------------
Vai solicitar autenticação via token toda vez que for executado

'''
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
import unicodedata

'''
ACESSO AO GOOGLE DRIVE
--------------------------
Vai solicitar autenticação via token toda vez que for executado

'''

# Abre o roda o Gdrive - Dá pra fazer pela barra lateral
from google.colab import drive


# Funções

In [28]:
'''
FUNÇÕES
-------
'''

def PrimLinhaCab(df):
  df.columns = df.iloc[0]
  df=df[1:]
  return df
  
def RemoveAcentos(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def ProcuraFuzzy(nome,serie):
  nome=str(nome)
  lista=list(serie)
  estrato=process.extractOne(nome,lista)
  nome=estrato[0]
  coef=estrato[1]
  posição=nome[2]
  return nome,coef,posição

def LimparNumeros(df):
  heads=df.columns.tolist()
  for head in heads:
    df[head]=df[head].str.replace(',', '.')
    df[head]=df[head].str.replace('%', '')
  return df  

def Emfloat(df,heads_s):
  heads=df.columns.tolist()
  head_n_str=[item for item in heads if item not in heads_s]
  dicio={}

  for head in head_n_str:
    dicio[head]=df[head].astype(float)
  return df

def RetornaDFS(nomes_planilhas,path):
  dfs = {}

  for planilha in nomes_planilhas:
    df_nome=planilha.replace(' ','_') # retira espaços
    df_nome=df_nome.replace('.','')#retira o ponto no final
    df_nome=df_nome.replace('-','_')#retira o ponto no final
    df_nome='df_'+df_nome
    dfs[df_nome] = pd.read_excel(path, sheet_name=planilha)

  return dfs

def LerExcel(path):
  xls = pd.ExcelFile(path) #abre o arquivo com todas as planilhas
  nomes_planilhas = xls.sheet_names # Uma lista das planilhas
  return nomes_planilhas

def FuzzyCoef(a,b,printar=False,testar=False):
  r=fuzz.ratio(a, b)
  pr=fuzz.partial_ratio(a, b)
  tsor=fuzz.token_sort_ratio(a, b)
  tser=fuzz.token_set_ratio(a, bytearray)
  n_r=0
  n_pr=0
  n_tsor=0
  n_tser=0
  if testar==True:
    if r>50:
      n_r=1
    if pr>50:
      n_pr=1
    if tsor>50:
      n_tsor=1
    if tser>50:
      n_ter=1
    if n_r + n_pr + n_tsor + n_tser <3:
      return False
    else:
      return True
  if printar==True:
    print(
        'Ratio:',r,'\n'
        'Partial Ration:',pr,'\n'
        'Token Sort Ratio:',tsor,'\n'
        'Token Set Ration',tser,'\n'
          )
    
def Fuzzy_Procv(df1,df2,key1,key2,key3=None):

  lista1=[]
  lista2=[]

  if key3 != None:
    for linha in df1[key1]:
      nome2,coef2,posi2=process.extractOne(linha,df2[key2])
      if coef2<100:
        resultado=FuzzyCoef(linha,nome2)
        if resultado:
          lista1.append(nome2)
          lista2.append(df2[key3][posi2])
        else:
          lista1.append('')
          lista2.append('')
      else:
        lista1.append(nome2)
        lista2.append(df2[key3][posi2])

  #return lista1
  df1['Match']=lista1
  df1['Nota2']=lista2

  return df1

def Minuscula(df,key):
  df[key]=df[key].astype(str).str.lower()
  return df
df_2020_1=Minuscula(df_2020_1,'Nome')

def Fuzzy_Procv_Valida(df1,df2,key1,key2,key3=None):
  index_valor=df2.set_index(key2)[key3] # Retorna série: vator de busca como indice e vetor retorno.
  df1[key3]=df1[key1].map(index_valor) # mapeia o nova série pelo indice e se encontrado retorna valor. 
  return df1v

In [5]:
# Vai solicitar autenticação via token toda vez que for executado
drive.mount('/content/drive/') # Monta o drive
!ls "/content/drive/My Drive/Dados" # Lista os arquivos da pasta

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
'dados FAPESP (26_05_2020).xlsx'	   'Inscritos no EUF 2020-1.gsheet'
'Histórico de Inscritos 2012-2020.gsheet'  'Notas EUF.gsheet'
'Histórico de Inscritos 2012-2020.xlsx'    'Notas EUF.xlsx'
'Inscritos na Pós.gsheet'		    resultado.xlsx
'Inscritos na Pós.xlsx'			    Teste.ipynb


In [7]:
path1 = '/content/drive/My Drive/Dados/Notas EUF.xlsx' # Caminho do arquivo
path2 = '/content/drive/My Drive/Dados/Inscritos na Pós.xlsx' # Caminho do arquivo

In [ ]:
path ='/content/drive/My Drive/Dados/'
arq = ['Notas EUF.xlsx', 'Inscritos na Pós.xlsx']
#dfs = {p: pd.read_csv(p) if p.endswith('.csv') else pd.read_excel(p) for p in ps}

In [15]:
nomes_planilhas=LerExcel(path1)
EUF_notas=RetornaDFS(nomes_planilhas,path1)
locals().update(EUF_notas) # tranforma dicionário em variável

nomes_planilhas2=LerExcel(path2)
Inscritos=RetornaDFS(nomes_planilhas2,path2)
locals().update(Inscritos)

In [32]:
#reset -f # Limpra variaveis
#dir() # will give you the list of in scope variables:
#globals() # will give you a dictionary of global variables
#locals() # will give you a dictionary of local variables
%whos DataFrame

Variable            Type         Data/Info
------------------------------------------
df_2020_1           DataFrame                             <...>n\n[62 rows x 17 columns]
df_EUF_2020_1       DataFrame             EUF CODE        <...>\n[722 rows x 19 columns]
df_Ins_EUF_2020_1   DataFrame                             <...>\n[952 rows x 33 columns]


In [16]:
df_2020_1

,Nome,curso no qual o candidato está se inscrevendo,Número de Inscrição,Instituição de onde vem,Email,Declaro que,Selecione o EUF cuja nota você pretende utilizar para ingresso,Nota antiga do Exame EUF,Orientador Informado,Tem carta de aceite?,Orientador Credenciado?,Estará credenciado na data de matrícula?,Observações,Data de Envio
0,Yuri Dias França,inscricaomd,7986,UFRJ,yuridiasf@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Aluno afirmou ter dissertação já pronta; desej...,22/06/2020 - 00:01
1,Gabriel Goulart Dimas,inscricaomd,7985,UFF,gabrielgd@id.uff.br,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Não ficou claro quando cola grau.,21/06/2020 - 23:55
2,Wellington Luiz dos Santos Araujo,incricaodr,7983,UFMA,wellingtonluiztj@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 23:08
3,Marcos Vinicius de Moraes,incricaodr,7982,UEM,marcos_moraes_1997@hotmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 23:03
4,André Farinha Bósio,incricaodr,7980,UEM,farinha96br@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 22:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Adriane Maria de Carvalho,inscricaomd,7845,UECE,adriane.carvalho@aluno.uece.br,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,09/06/2020 - 17:23
58,Carlos Alberto Silva,inscricaomd,7844,UNIFEI,carlosatibaia@uol.com.br,Não desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,09/06/2020 - 17:21
59,Alan Müller,inscricaomd,7840,IFUSP,alan.muller26@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Deve colar grau até início de 08/2020.,09/06/2020 - 03:25
60,Thiago de Lima Campos,inscricaomd,7839,ME no IFUSP,tdlimacampos@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,Carlos Molina Mendes,Nao,Sim,Sim,"Já está no ME, deseja concorrer à bolsa. Será ...",08/06/2020 - 21:35


In [ ]:
# Metodo de importar arquivo GOOGLE DOCS (Sheet)
# Planilha das notas do EUF
url1 = 'https://docs.google.com/spreadsheets/d/1cgTJ2hxn4U2XUX_LRW_8I427mZ6s2tDzc99VVRJZaI8/edit#gid=0'
wb1 = gc.open_by_url(url1)

# Planilha das inscrições
url2='https://docs.google.com/spreadsheets/d/1CEXvVbvbWEsjg25lcfiHauqGAaBcwgZeggkudD8KQFM/edit#gid=0'
wb2 = gc.open_by_url(url2)

sheet1 = wb1.worksheet('EUF 2020-1')
sheet2 = wb2.worksheet('2020-1')
data1 = sheet1.get_all_values()
data2 = sheet2.get_all_values()
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

WorksheetNotFound: ignored

In [ ]:
df1=PrimLinhaCab(df1)
df2=PrimLinhaCab(df2)

heads_s=['EUF CODE','Nome','Doc',"Nº Doc"]

#LimparNumeros(df1)

df1=Emfloat(df1,heads_s)

# METADADOS
df1.attrs={
    'Base':'EUF 2020-1',
    'Referência':'N/A',
    'Origem':'Plataforma EUF'
    }

# METADADOS
df2.attrs={
    'Base':'EUF 2020-1',
    'Referência':'N/A',
    'Origem':'CPGpar'
    }

ValueError: ignored

# Ajeitando DataFrames
---

In [ ]:
heads=df.columns.tolist()
heads_strings=['EUF CODE','Nome','Doc',"Nº Doc"]
head_n_str=[item for item in heads if item not in heads_strings]
dicio={}

for head in head_n_str:
  dicio[head]=df[head].astype(float)


# Testes com Fuzzy
---

In [43]:
a='Carlos Alberto Silva	'
b='Alexander Kopernik da Silva Guerreiro'
c='Carlos	João'
d='Carlos Kopernik da Silva Guerreiro'

In [ ]:
lista2=df1['Nome'].apply(lambda x: process.extractOne(x,df2['Nome']))
lista2

In [98]:
# df=df.drop(columns=['Apto','Nota Normalizada'])
df2[1][60]

NameError: ignored

In [107]:
df_2020_1['Nome'].apply(lambda x: x.astype(str).str.lower())


AttributeError: ignored

In [20]:
def Minuscula(df,key):
  df[key]=df[key].astype(str).str.lower()
  return df
df_2020_1=Minuscula(df_2020_1,'Nome')

In [25]:
def Fuzzy_Procv_Valida(df1,df2,key1,key2,key3=None):
  index_valor=df2.set_index(key2)[key3] # Retorna série: vator de busca como indice e vetor retorno.
  df1[key3]=df1[key1].map(index_valor) # mapeia o nova série pelo indice e se encontrado retorna valor. 
  return df1

In [29]:
Fuzzy_Procv_Valida(df_2020_1,df_EUF_2020_1,'Nome','Nome','Nota')

,Nome,curso no qual o candidato está se inscrevendo,Número de Inscrição,Instituição de onde vem,Email,Declaro que,Selecione o EUF cuja nota você pretende utilizar para ingresso,Nota antiga do Exame EUF,Orientador Informado,Tem carta de aceite?,Orientador Credenciado?,Estará credenciado na data de matrícula?,Observações,Data de Envio,Nota
0,yuri dias frança,inscricaomd,7986,UFRJ,yuridiasf@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Aluno afirmou ter dissertação já pronta; desej...,22/06/2020 - 00:01,4.00
1,gabriel goulart dimas,inscricaomd,7985,UFF,gabrielgd@id.uff.br,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Não ficou claro quando cola grau.,21/06/2020 - 23:55,3.00
2,wellington luiz dos santos araujo,incricaodr,7983,UFMA,wellingtonluiztj@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 23:08,2.75
3,marcos vinicius de moraes,incricaodr,7982,UEM,marcos_moraes_1997@hotmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 23:03,4.25
4,andré farinha bósio,incricaodr,7980,UEM,farinha96br@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 22:23,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,adriane maria de carvalho,inscricaomd,7845,UECE,adriane.carvalho@aluno.uece.br,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,09/06/2020 - 17:23,3.25
58,carlos alberto silva,inscricaomd,7844,UNIFEI,carlosatibaia@uol.com.br,Não desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,09/06/2020 - 17:21,NaN
59,alan müller,inscricaomd,7840,IFUSP,alan.muller26@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Deve colar grau até início de 08/2020.,09/06/2020 - 03:25,5.50
60,thiago de lima campos,inscricaomd,7839,ME no IFUSP,tdlimacampos@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,Carlos Molina Mendes,Nao,Sim,Sim,"Já está no ME, deseja concorrer à bolsa. Será ...",08/06/2020 - 21:35,5.25


In [ ]:
def FuzzyMatch(lista1,lista2):
  lista=[]
  lista_nome=[]
  for linha in lista1:
    lista.append(process.extractOne(linha, lista2)) # Retorna o nome, coeficiente, index no pandas
    lista_nome.append(linha)
  return lista

In [30]:
def Fuzzy_Procv(df1,df2,key1,key2,key3=None):

  #lista1=df2[key2].tolist()
  #lista2=df1[key1].apply(lambda x: process.extractOne(x,lista))
  #lista3=[]

  lista1=[]
  lista2=[]

  if key3 != None:
    for linha in df1[key1]:
      nome2,coef2,posi2=process.extractOne(linha,df2[key2])
      if coef2<100:
        resultado=FuzzyCoef(linha,nome2)
        if resultado:
          lista1.append(nome2)
          lista2.append(df2[key3][posi2])
        else:
          lista1.append('')
          lista2.append('')
      else:
        lista1.append(nome2)
        lista2.append(df2[key3][posi2])

  #return lista1
  df1['Match']=lista1
  df1['Nota2']=lista2

  return df1



In [31]:
Fuzzy_Procv(df_2020_1,df_EUF_2020_1,'Nome','Nome','Nota')

,Nome,curso no qual o candidato está se inscrevendo,Número de Inscrição,Instituição de onde vem,Email,Declaro que,Selecione o EUF cuja nota você pretende utilizar para ingresso,Nota antiga do Exame EUF,Orientador Informado,Tem carta de aceite?,Orientador Credenciado?,Estará credenciado na data de matrícula?,Observações,Data de Envio,Nota,Match,Nota2
0,yuri dias frança,inscricaomd,7986,UFRJ,yuridiasf@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Aluno afirmou ter dissertação já pronta; desej...,22/06/2020 - 00:01,4.00,yuri dias frança,4
1,gabriel goulart dimas,inscricaomd,7985,UFF,gabrielgd@id.uff.br,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Não ficou claro quando cola grau.,21/06/2020 - 23:55,3.00,gabriel goulart dimas,3
2,wellington luiz dos santos araujo,incricaodr,7983,UFMA,wellingtonluiztj@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 23:08,2.75,wellington luiz dos santos araujo,2.75
3,marcos vinicius de moraes,incricaodr,7982,UEM,marcos_moraes_1997@hotmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 23:03,4.25,marcos vinicius de moraes,4.25
4,andré farinha bósio,incricaodr,7980,UEM,farinha96br@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,21/06/2020 - 22:23,4.00,andré farinha bósio,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,adriane maria de carvalho,inscricaomd,7845,UECE,adriane.carvalho@aluno.uece.br,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,09/06/2020 - 17:23,3.25,adriane maria de carvalho,3.25
58,carlos alberto silva,inscricaomd,7844,UNIFEI,carlosatibaia@uol.com.br,Não desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,NaN,09/06/2020 - 17:21,NaN,,
59,alan müller,inscricaomd,7840,IFUSP,alan.muller26@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,NaN,Nao,NaN,NaN,Deve colar grau até início de 08/2020.,09/06/2020 - 03:25,5.50,alan müller,5.5
60,thiago de lima campos,inscricaomd,7839,ME no IFUSP,tdlimacampos@gmail.com,Desejo concorrer à Bolsa,NaN,NaN,Carlos Molina Mendes,Nao,Sim,Sim,"Já está no ME, deseja concorrer à bolsa. Será ...",08/06/2020 - 21:35,5.25,thiago de lima campos,5.25


In [ ]:
lista=df1['Nome'].tolist()
lista1=df2['Nome'].tolist()

for linha in lista:
  nome,coef,posi=process.extractOne(linha, lista1)
  print(nome,coef,posi)

In [ ]:
df1['Novo']=df2['Nome'].apply(lambda x: process.extractOne(x))

In [ ]:
df1=
head1=
df2=
head2=
head_alvo=
lista=[]
for linha in lista1:
  match=process.extractOne(linha, lista2)
  for x in match:
    nome=match[0]
    coef=match[1]
    posição=match[2]
  lista_append(df2.head_alvo[coef])

In [ ]:
lista=process.extract("andre", df2['Nome'], limit=1)
for nome in lista:
    coef=nome[1]
    posição=nome[2]
return 

In [ ]:
# Correção dos variáveis dentro do Dataframe do Pandas
# Usa paenas se o código acima não funcionar bem

df.columns
df_novo = {
'Neighborhood Group': df['neighbourhood_group'].astype(str),
'Neighborhood': df['neighbourhood'].astype(str),
'Latitude': df['latitude'].astype(float),
'Longitude': df['longitude'].astype(float),
'Room Type': df['room_type'].astype(str),
'Price': df['price'].astype(float),
'Minimum Nights': df['minimum_nights'].astype(int),
'Number of Reviews': df['number_of_reviews'].astype(int),
'Reviews Per Month': df['reviews_per_month_new'].astype(float)
}

In [ ]:
media=df['Nota'].mean()
df['Nota Normalizada'] = df.apply(lambda linha: linha['Nota'] * 5/media, axis=1)

In [ ]:
df_bonus = pd.read_excel('/content/drive/My Drive/Dados/Notas EUF.xlsx')

In [ ]:
# MULTIPLAS CONDIÇÕES PARA CLASSIFICAR UMA COLUNA 
conditions = [
    (df['Set'] == 'Z') & (df['Type'] == 'A'),
    (df['Set'] == 'Z') & (df['Type'] == 'B'),
    (df['Type'] == 'B')]
choices = ['yellow', 'blue', 'purple']
df['color'] = np.select(conditions, choices, default='black')
print(df)

In [ ]:
df_bonus['Nota']

In [ ]:
a=22/7
a

In [ ]:
import math
math.pi

# Referências

https://medium.com/analytics-vidhya/colab-and-google-sheets-surprisingly-powerful-combination-for-data-science-part-1-bbbb11cbd8e

https://stackoverflow.com/questions/19913659/pandas-conditional-creation-of-a-series-dataframe-column

https://stackoverflow.com/questions/37926248/how-to-remove-accents-from-values-in-columns

https://stackoverflow.com/questions/48376580/google-colab-how-to-read-data-from-my-google-drive

https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92#:~:text=You%20should%20see%20a%20screen,select%20%E2%80%9CGet%20shareable%20link%E2%80%9D.